# Klasyfikacja wieloetykietowa

![multilabel_intro.png](https://live.staticflickr.com/65535/54927204610_28ff033051.jpg)

*Obraz wygenerowany przez narzędzie generowania obrazów ChatGPT.*

## Wstęp

Współczesne zdjęcia często zawierają wiele obiektów i złożone sceny. Przykładem jest fotografia powyżej: widzimy na niej mężczyznę w marynarce, koszuli i krawacie, kobietę w sukni z welonem, grupę osób w letnich ubraniach, a całość rozgrywa się na plaży, na tle wody i zachodu słońca.

Gdybyśmy trzymali się klasycznego podejścia „jedno zdjęcie — jedna etykieta”, musielibyśmy wybrać tylko jedną kategorię: czy to zdjęcie przedstawia ślub, mężczyznę, kobietę, marynarkę, suknię, plażę…? W praktyce oznacza to sztuczne ograniczenie informacji.

Nie musimy jednak ograniczać się do jednej etykiety. Właśnie dlatego stosuje się klasyfikację wieloetykietową, która pozwala przypisać jednemu obrazowi wiele kategorii opisujących różne obiekty.
Zamiast wybierać jedną etykietę, możemy więc powiedzieć:
„Na tym zdjęciu występują instancje klas: człowiek, marynarka, koszula, krawat, suknia, plaża itd.”

## Zadanie

Twoim zadaniem jest zdefiniowanie i wytrenowanie sieci neuronowej do realizacji klasyfikacji wieloetykietowej. W zadaniu kompozycja zdjęcia będzie uproszczona w stosunku do przykładu powyżej. Na zdjęciach będą widoczne tylko ubrania, a Twoim zadaniem jest stworzenie modelu, który potrafi określić, czy dany rodzaj ubrania występuje na zdjęciu.

## Dane

W zadaniu masz do dyspozycji 
* zbiór treningowy ($6318$ próbek),
* zbiór walidacyjny ($702$ próbek). 

Zbiór testowy, na którym finalnie będzie oceniane Twoje rozwiązanie ma 780 próbek i jest niejawny. Został on stworzony w ten sam sposób co zbiór walidacyjny więc ma analogiczną charakterystykę.

Każda próbka to obraz o wymiarach $168 \times 168$ pikseli. Z każdym obrazkiem skojarzony jest 10-elementowy wektor z wartościami $0$ i $1$, który określa obecność danej klasy na obrazku. Informacja o tym jakiej części garderoby odpowiada poszczególny indeks w wektorze etykiet jest podana w słowniku `LABEL_NAMES` zdefiniowanym w jednej z komórek z kodem.

## Kryterium Oceny
Ostateczna ocena zadania będzie na podstawie średniej wartości miary $F1$ policzonej w ramach schematu *macro*.

Za to zadanie możesz zdobyć pomiędzy 0 a 100 punktów. Twój finalny wynik punktowy za rozwiązanie zadania obliczony będzie według poniższej funkcji (im wyższa wartość tym lepiej) przy dodatkowym zastosowaniu zaokrąglenia do wartości całkowitych:
$$
\mathrm{score} =
\begin{cases}
    0 & \text{jeżeli } {F1}\leq 0.57 \\
    100 \times \frac{{F1}- 0.57}{0.87 - 0.57} & \text{jeżeli } 0.57 < {F1} < 0.87 \\
    100 & \text{jeżeli } {F1} \geq 0.87
\end{cases}
$$

## Ograniczenia

- Twoje rozwiązanie będzie testowane na Platformie Konkursowej w środowisku z GPU. Na Platformie nie ma dostępu do Internetu, jednak możliwe jest skorzystanie z pretrenowanych modeli ResNet (*ResNet18*, *ResNet34*, *ResNet50*) z pakietu torchvision, które są zapisane w pamięci w formie plików. Aby z nich skorzystać, należy zastosować taką samą komendę w kodzie jak w przypadkach gdy Internet jest dostępny.
- Ewaluacja Twojego finalnego rozwiązania na danych testowych na Platformie Konkursowej nie może trwać dłużej niż 2.5 minuty z GPU.

## Pliki zgłoszeniowe

Ten notebook uzupełniony o Twoje rozwiązanie (definicję modelu, funkcję do treningu modelu i funkcję zwracającą predykcje modelu).

## Ewaluacja

Pamiętaj, że podczas sprawdzania flaga `FINAL_EVALUATION_MODE` zostanie ustawiona na `True`.

Za to zadanie możesz zdobyć pomiędzy 0 a 100 punktów. Liczba punktów, którą zdobędziesz, będzie wyliczona na (tajnym) zbiorze testowym na Platformie Konkursowej na podstawie wyżej wspomnianego wzoru, zaokrąglona do liczby całkowitej. Jeśli Twoje rozwiązanie nie będzie spełniało powyższych kryteriów lub nie będzie wykonywać się prawidłowo, otrzymasz za zadanie 0 punktów.

## Kod Startowy
W tej sekcji inicjalizujemy środowisko poprzez zaimportowanie potrzebnych bibliotek i funkcji. Przygotowany kod ułatwi Tobie efektywne operowanie na danych i budowanie właściwego rozwiązania.

In [ ]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI PODCZAS WYSYŁANIA ##########################

FINAL_EVALUATION_MODE: bool = False  # Podczas sprawdzania ustawimy tę flagę na True.

In [ ]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI PODCZAS WYSYŁANIA ##########################

import os

import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as fun
import torchvision

from torch.utils.data import DataLoader, TensorDataset
from torchvision import transforms

from tqdm import tqdm

from sklearn.metrics import f1_score

In [ ]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI PODCZAS WYSYŁANIA ##########################

def seed_everything(seed: int):
    """Ustawia ziarno (seed) dla reprodukowalności wyników w Pythonie, NumPy oraz PyTorch."""

    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [ ]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI PODCZAS WYSYŁANIA ##########################

# liczba klas do klasyfikacji
N_CLASSES: int = 10

# mapowanie indeksu klasy na jej nazwę
LABEL_NAMES: dict[int, str] = {
    0: "T-shirt/top",
    1: "Trouser",
    2: "Pullover",
    3: "Dress",
    4: "Coat",
    5: "Sandal",
    6: "Shirt",
    7: "Sneaker",
    8: "Bag",
    9: "Ankle boot"
}

In [ ]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI PODCZAS WYSYŁANIA ##########################

if not FINAL_EVALUATION_MODE:
    FILES: list[str] = [
        "runway-mnist/train-x.npz",
        "runway-mnist/train-y.npz",
        "runway-mnist/val-x.npz",
        "runway-mnist/val-y.npz"
    ]

    # pobierz dane na nowo, jeśli czegoś brakuje
    if not all(os.path.exists(file) for file in FILES):
        import gzip
        import tarfile
        import shutil

        if not os.path.exists("runway-mnist"):
            os.mkdir("runway-mnist")

        COMPRESSED_ARCHIVE = "runway-mnist.tar.gz"
        TAR_ARCHIVE  = COMPRESSED_ARCHIVE.rstrip(".gz")
        DOWNLOAD_URL = "https://drive.google.com/uc?id=1oNAFYdJyCVe3Po90KLUPxAG9HGuL7NSw"

        try:
            import gdown
        except ImportError as err:
            raise RuntimeError("Do pobrania zbioru danych potrzebujesz lokalnej instalacji pakietu gdown: `pip install gdown`") from err

        gdown.download(DOWNLOAD_URL, str(COMPRESSED_ARCHIVE), quiet=False)

        with gzip.open(COMPRESSED_ARCHIVE, "rb") as compressed:
            with open(TAR_ARCHIVE, "wb") as archive:
                shutil.copyfileobj(compressed, archive)

        os.remove(COMPRESSED_ARCHIVE)
        print(f"Zdekompresowano: {TAR_ARCHIVE}")

        with tarfile.open(TAR_ARCHIVE, "r") as tar:
            tar.extractall("runway-mnist")

        os.remove(TAR_ARCHIVE)
        print(f"Rozpakowano: {TAR_ARCHIVE}")

### Ładowanie danych

In [ ]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI PODCZAS WYSYŁANIA ##########################

SEED: int = 42
seed_everything(SEED)

def load_x(usage) -> torch.Tensor:
    path = f"runway-mnist/{usage}-x.npz"
    return torch.tensor(np.load(path)["images"], dtype=torch.float32).unsqueeze(1)

def load_y(usage) -> torch.Tensor:
    path = f"runway-mnist/{usage}-y.npz"
    return torch.tensor(np.load(path)["labels"], dtype=torch.long)

train_dataset = TensorDataset(load_x("train"), load_y("train"))
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

val_dataset = TensorDataset(load_x("val"), load_y("val"))
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

In [ ]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI PODCZAS WYSYŁANIA ##########################

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

## Funkcja oceniająca

In [ ]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI PODCZAS WYSYŁANIA ##########################

def compute_score(f1: float) -> int:
    """Oblicza wynik punktowy na podstawie wartości metryki F1."""
    lower_bound = 0.57
    upper_bound = 0.87

    if f1 <= lower_bound:
        return 0
    elif lower_bound < f1 < upper_bound:
        return int(round(100 * (f1 - lower_bound) / (upper_bound - lower_bound)))
    else:
        return 100

def evaluate_algorithm(model, predict, loader) -> float:
    """Oblicza metryki oraz ocenia Twoje rozwiązanie na ich podstawie. Zwraca obliczoną wartość metryki F1."""
    preds = []
    labels = []
    model.eval()
    with torch.no_grad():
        for x, y in loader:
            prediction = predict(model, x.to(device)).cpu()
            preds.append(prediction)
            labels.append(y)

    predictions = torch.cat(preds)
    labels = torch.cat(labels)

    f1 = f1_score(labels.numpy(), predictions.numpy(), average="macro")
    points = compute_score(f1)
    print(f"Twój wynik F1: {f1:.3f}, co daje {points} punktów.")
    return f1

## Przykładowe rozwiązanie

Poniżej przedstawiamy uproszczone rozwiązanie, które demonstruje podstawową funkcjonalność
notebooka. Może ono posłużyć jako punkt wyjścia do opracowania Twojego rozwiązania.

In [ ]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI PODCZAS WYSYŁANIA ##########################

class NaiveSolution(nn.Module):
    """Rozwiązanie naiwne."""

    def __init__(self):
        super().__init__()

    def forward(self, input: torch.Tensor) -> torch.Tensor:
        """Ten naiwny model przewiduje, że wszystkie klasy są na obrazku."""
        BATCH_SIZE = input.size(0)
        return torch.Tensor([1] * BATCH_SIZE * N_CLASSES).reshape(BATCH_SIZE, -1)

def train_naive(_: NaiveSolution):
    """Model, nie wymaga treningu."""
    pass

def predict_naive(model: NaiveSolution, input: torch.Tensor) -> torch.Tensor:
    """Nasz model zwraca od razu predykcje modelu, nie przetwarzamy ich dodatkowo."""
    return model(input).to(torch.long)

In [ ]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI PODCZAS WYSYŁANIA ##########################
if not FINAL_EVALUATION_MODE:
    naive_solution = NaiveSolution().to(device)
    naive_solution.train()
    train_naive(naive_solution)
    evaluate_algorithm(naive_solution, predict_naive, val_loader)

## Twoje rozwiązanie
W komórce poniżej należy umieścić Twoje rozwiązanie. Wprowadzaj zmiany wyłącznie tutaj!

In [ ]:
class Solution(nn.Module):
    """Twoje rozwiązanie."""

    def __init__(self):
        super().__init__()

    def forward(self, input: torch.Tensor) -> torch.Tensor:
        """Inferencja modelu"""
        BATCH_SIZE = input.size(0)
        return torch.rand(BATCH_SIZE * N_CLASSES).reshape(BATCH_SIZE, -1)

def train_solution(_: Solution):
    """Pętla treningowa dla Twojego modelu."""
    pass

def predict_solution(model: Solution, input: torch.Tensor) -> torch.Tensor:
    """Klasyfikacja z użyciem modelu.
    Wydzielenie tej funkcji ma na celu proste umożliwienie post-processing'u wyników z modelu."""
    predictions = model(input).round().to(torch.long)
    return predictions

In [ ]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI PODCZAS WYSYŁANIA #########################

solution = Solution().to(device)
solution.train()

train_solution(solution)

## Ewaluacja

Poniższy kod będzie służył ewaluacji rozwiązania. Po wysłaniu rozwiązania do nas zostanie wykonana funkcja `evaluate_algorithm(solution, predict_solution)`, t.j. prawie identyczny kod jak niżej będzie się uruchamiał na zbiorze testowym dostępnym tylko dla sprawdzających zadania.

Upewnij się przed wysłaniem, że cały notebook wykonuje się od początku do końca bez błędów i bez ingerencji użytkownika po wykonaniu polecenia `Run All`.

In [ ]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI PODCZAS WYSYŁANIA #########################

if not FINAL_EVALUATION_MODE:
    evaluate_algorithm(solution, predict_solution, val_loader)